In [1]:
import pandas as pd
import numpy as np

In [2]:
# download
pd_image_norm = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/paths_to_channels_creation/dmso_stats_v1v2.csv')

In [44]:
np.empty(shape = (1, 5))

array([[4.9e-324, 9.9e-324, 1.5e-323, 2.0e-323, 2.5e-323]])

In [3]:
# assess
pd_image_norm.head(3)

,Unnamed: 0,P015076,P015076.1,P015076.2,P015076.3,P015076.4,P015077,P015077.1,P015077.2,P015077.3,...,P101032,P101032.1,P101032.2,P101032.3,P101032.4,P101033,P101033.1,P101033.2,P101033.3,P101033.4
0,NaN,C1,C2,C3,C4,C5,C1,C2,C3,C4,...,C1,C2,C3,C4,C5,C1,C2,C3,C4,C5
1,m,298.56553513638613,1182.599314422301,1740.3659357495999,1364.0816585652478,682.3957518471755,333.64529227135034,1350.6730676892814,2163.296099828229,1633.245623894769,...,768.9873166047475,939.7879726593701,2398.1907485384286,1741.027277214412,2286.808505239005,857.7293221922147,1140.2277511119682,2201.8708363130613,1827.0072842805962,2341.0384757127335
2,std,280.17719870419074,1628.216043824049,846.2971853421564,1029.1825133612128,704.7482234564715,339.08582350494197,1606.9584977581317,1011.646483085695,1160.5192705314935,...,1345.487814435877,1080.628213403506,1735.0736625552915,2178.6846020690264,1496.4279014680417,1353.2188428308662,1246.2801113854985,1516.8765772351915,1987.989843616199,1461.345761315124


In [28]:
plate = 'P015076'
channel = "C2"

In [29]:
if channel == "C1":
    extract = plate
elif channel == "C2":
    extract = plate + '.1'
elif channel == "C3":
    extract = plate + '.2'
elif channel == "C4":
    extract = plate + '.3'
else:
    extract = plate + '.4'

In [30]:
extract

'P015076.1'

In [33]:
ans = pd_image_norm[extract]

In [34]:
ans

0                   C2
1    1182.599314422301
2    1628.216043824049
Name: P015076.1, dtype: object

In [40]:
mean = float(ans.iloc[1])

In [41]:
mean

1182.599314422301

# Plan
1. Use splitting script to split compounds into test, validation and training data
2. Extract the compounds for those and use them to group indices for picture data and chemical structure data
3. Take those indices and adjust the EfficientNet and Chemical Structure Torch models 
4. Evaluate performance

# 1

In [5]:
paths_v1v2 = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/paths_to_channels_creation/paths_channels_treated_v1v2.csv')

In [16]:
paths_v1v2.columns

Index(['Unnamed: 0', 'plate', 'well', 'compound', 'C1', 'C2', 'C3', 'C4', 'C5',
       'moa'],
      dtype='object')

In [21]:
# testing using pandas dataframe
training_set = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/CS_data_splits/CS_training_set_cyclo_adr_2.csv')
validation_set = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/CS_data_splits/CS_valid_set_cyclo_adr_2.csv')
test_set = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/data_for_models/CS_data_splits/CS_test_set_cyclo_adr_2.csv')

In [22]:
# check if it seems reasonable
test_set.head()

,Unnamed: 0,Compound_ID,Batch_nr,CUSTOMER_ID,MOLFORMULA,MOLWEIGHT,NAME,VENDOR,ADD_INFO,SMILES,...,AC50,EFFICACY,REPRODUCIBILITY,CURVE_RANK,FLAG,CAS,PUBCHEM_CID,PUBCHEM_SID,PURITY_RATING,PURITY_RATING_4M
0,20,CBK308624,BJ1896979,A19458515,C46H58ClN5O8,844.46,Proglumetacin,AK Scientific,57132-53-3,CCCN(CCC)C(=O)C(CCC(=O)OCCCN1CCN(CCOC(=O)Cc2c(...,...,7.555014,79.193028,active_match,6.0,NaN,57132-53-3,4921.0,144206876.0,A,A
1,213,CBK041670,BJ1895140,A22844106,C13H11N3O4S2,337.38,TENOXICAM,Microsource Discovery Systems,59804-37-4,CN1C(C(=O)Nc2ccccn2)C(=O)c2sccc2S1(=O)=O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,232,CBK309288,BJ1897859,K19515365,C16H14N2O3,282.30,bendazac,Key Organics Ltd.,NaN,OC(=O)COc1nn(Cc2ccccc2)c2ccccc12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,239,CBK201028,BJ1897386,K54770957,C18H15ClN2O2S,358.85,Etoricoxib,MedChemExpress,"CAS 202409-33-4, target COX",O=S(C1=CC=C(C2=CC(Cl)=CN=C2C3=CC=C(C)N=C3)C=C1...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,400,CBK004363,BJ1894494,K41524689,C8H9NO2,151.17,ACETAMINOPHEN,Microsource Discovery Systems,103-90-2,CC(=O)Nc1ccc(O)cc1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2

In [23]:
# extract compound IDs
test_data_lst= list(test_set["Compound_ID"].unique())
train_data_lst= list(training_set["Compound_ID"].unique())
valid_data_lst= list(validation_set["Compound_ID"].unique())

In [24]:
# check if works
print(train_data_lst)

['CBK290981', 'CBK041553', 'CBK011692', 'CBK290941', 'CBK200996', 'CBK013361', 'CBK011551', 'CBK011617', 'CBK011647', 'CBK011672', 'CBK011684', 'CBK011685', 'CBK015987', 'CBK016153', 'CBK016300', 'CBK018231', 'CBK024260', 'CBK041382', 'CBK041439', 'CBK041512', 'CBK041537', 'CBK041555', 'CBK041690', 'CBK041694', 'CBK041905', 'CBK042102', 'CBK200528', 'CBK200564', 'CBK200655', 'CBK200972', 'CBK200974', 'CBK201004', 'CBK289991', 'CBK290002', 'CBK290013', 'CBK290208', 'CBK290222', 'CBK290503', 'CBK290707', 'CBK307870', 'CBK042135', 'CBK011697', 'CBK023992', 'CBK042165', 'CBK308569', 'CBK011674', 'CBK041403', 'CBK024259', 'CBK200526', 'CBK200707', 'CBK200900', 'CBK201000', 'CBK308605', 'CBK308582', 'CBK041550', 'CBK041548', 'CBK200939', 'CBK308602', 'CBK308915', 'CBK290018', 'CBK200956']


In [25]:
training_set = paths_v1v2[paths_v1v2["compound"].isin(train_data_lst)].reset_index(drop=True)
validation_set = paths_v1v2[paths_v1v2["compound"].isin(valid_data_lst)].reset_index(drop=True)
test_set = paths_v1v2[paths_v1v2["compound"].isin(test_data_lst)].reset_index(drop=True)

In [26]:
print(training_set.shape)
training_set.head()

(3402, 10)


,Unnamed: 0,plate,well,compound,C1,C2,C3,C4,C5,moa
0,162,P015076,C10,CBK290981,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,cyclooxygenase inhibitor
1,163,P015076,C10,CBK290981,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,cyclooxygenase inhibitor
2,164,P015076,C10,CBK290981,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,cyclooxygenase inhibitor
3,165,P015076,C10,CBK290981,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,cyclooxygenase inhibitor
4,166,P015076,C10,CBK290981,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,/share/mikro/IMX/MDC_pharmbio/specs935-v1/spec...,cyclooxygenase inhibitor


In [1]:
import torch
state = torch.load('/home/jovyan/Tomics-CP-Chem-MoA/02_CP_Models/saved_models' +'/' + 'CP_least_loss_model')

In [2]:
 state["f1_score"]

0.9252695159756236

In [3]:
state["accuracy"]

0.9216524216524217

In [70]:
state["valid_loss"]

14.077402258210459

In [71]:
run['metrics/epochs'] = max_epochs

Exception: File `'[metrics/epochs].py'` not found.

In [63]:
import torch
state = torch.load('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/Best_Tomics_Model/saved_models' +'/' + 'Tomics_1DCNN')

In [64]:
 state["f1_score"]

0.5092811377791403

In [65]:
state["accuracy"]

0.4555635838150289

In [66]:
state["valid_loss"]

13.646594799875508